# HW 6

## Question 1

We wish to estimate the surprise number (2nd moment) of a data stream, using the method of AMS. It happens that our stream consists of ten different values, which we'll call 1, 2,..., 10, that cycle repeatedly. That is, at timestamps 1 through 10, the element of the stream equals the timestamp, at timestamps 11 through 20, the element is the timestamp minus 10, and so on. It is now timestamp 75, and a 5 has just been read from the stream. As a start, you should calculate the surprise number for this time.

For our estimate of the surprise number, we shall choose three timestamps at random, and estimate the surprise number from each, using the AMS approach (length of the stream times 2m-1, where m is the number of occurrences of the element of the stream at that timestamp, considering all times from that timestamp on, to the current time). Then, our estimate will be the median of the three resulting values.
You should discover the simple rules that determine the estimate derived from any given timestamp and from any set of three timestamps. Then, identify from the list below the set of three "random" timestamps that give the closest estimate.

**Solution**

In [54]:
from numpy import median

target = 5 * (8**2 + 7**2)

def suprise_number(x, y, z):
    mx = 75 * (2 * sum(1 for i in range(x, 76, 10)) - 1)
    my = 75 * (2 * sum(1 for i in range(y, 76, 10)) - 1)
    mz = 75 * (2 * sum(1 for i in range(z, 76, 10)) - 1)
    #print(mx, my, mz)
    return median([mx, my, mz])

n = [
    (31, 48, 50),
    (5, 33, 67),
    (37, 46, 55),
    (3, 45, 72)
]

print("real suprise number is", target)

for z in n:
    t = suprise_number(*z)
    print(z, "-> estimated suprise number is", t)
    
    

real suprise number is 565
(31, 48, 50) -> estimated suprise number is 375.0
(5, 33, 67) -> estimated suprise number is 675.0
(37, 46, 55) -> estimated suprise number is 375.0
(3, 45, 72) -> estimated suprise number is 525.0


## Question 2

Suppose we are using the DGIM algorithm of Section 4.6.2 to estimate the number of 1's in suffixes of a sliding window of length 40. The current timestamp is 100, and we have the following buckets stored: $[(100, 1),(98, 1),(95, 2),(92, 2),(87, 4),(80, 8),(65, 8)]$. Note: we are showing timestamps as absolute values, rather than modulo the window size, as DGIM would do.

Suppose that at times 101 through 105, 1's appear in the stream. Compute the set of buckets that would exist in the system at time 105. Then identify one such bucket from the list below. Buckets are represented by pairs (end-time, size).

**Solution**

In [51]:
b = [
    (100, 1),
    (98, 1),
    (95, 2),
    (92, 2),
    (87, 4),
    (80, 8),
    (65, 8)
]

def update(t):
    
    global b
    
    if t - b[-1][0] >= 40:
        b = b[:-1]
        
    b.insert(0, (t, 1))
    
    for i in [1, 2, 4, 8]:
        if list(c[1] for c in b).count(i) > 2:
            n = next(
                (k for k, c in enumerate(b) if c[1] == 2 * i), 
                len(b)
            )
            u = n - 1
            v = n - 2
            b[u] = (b[v][0], i * 2)
            b.pop(v)
            
    print(b)
    
    
update(101)
update(102)
update(103)
update(104)
update(105)

[(101, 1), (100, 2), (95, 4), (87, 4), (80, 8), (65, 8)]
[(102, 1), (101, 1), (100, 2), (95, 4), (87, 4), (80, 8), (65, 8)]
[(103, 1), (102, 2), (100, 2), (95, 4), (87, 4), (80, 8), (65, 8)]
[(104, 1), (103, 1), (102, 2), (100, 2), (95, 4), (87, 4), (80, 8), (65, 8)]
[(105, 1), (104, 2), (102, 4), (95, 8), (80, 8)]


## Question 3

We wish to use the Flagolet-Martin algorithm of Section 4.4 to count the number of distinct elements in a stream. Suppose that there are ten possible elements, 1, 2,..., 10, that could appear in the stream, but only four of them have actually appeared. To make our estimate of the count of distinct elements, we hash each element to a 4-bit binary number. The element x is hashed to 3x + 7 (modulo 11). For example, element 8 hashes to 3*8+7 = 31, which is 9 modulo 11 (i.e., the remainder of 31/11 is 9). Thus, the 4-bit string for element 8 is 1001.

A set of four of the elements 1 through 10 could give an estimate that is exact (if the estimate is 4), or too high, or too low. You should figure out under what circumstances a set of four elements falls into each of those categories. Then, identify in the list below the set of four elements that gives the exactly correct estimate.

**Solution**

In [52]:
f1 = lambda c: bin((3 * c + 7) % 11)[2:][-1::-1]
f2 = lambda c: next((i for i, v in enumerate(f1(c)) if v =="1"), 0)
f3 = lambda s: max(2**f2(k) for k in s)

v = [
    {1, 6, 7, 10},
    {2, 5, 7, 10},
    {1, 4, 7, 9},
    {2, 3, 6, 9}
]

for s in v:
    print(s, "distinct elements estimate =", f3(s))


{1, 10, 6, 7} distinct elements estimate = 4
{2, 10, 5, 7} distinct elements estimate = 4
{1, 4, 9, 7} distinct elements estimate = 8
{9, 2, 3, 6} distinct elements estimate = 2
